## Check where this runs
***

In [6]:
import os
os.environ["our_new"] = ""
envs = os.getenv("LOGNAME")
print(envs)
assert envs == "hughharford"
# if you get an error here, the notebook is not running in HSTH's expected place, and adjusts will be required.
print(os.environ)
if envs == "hughharford":
    environment_running = 'laptop'
else:
    environment_running = 0

hughharford
environ({'BUNDLER_EDITOR': 'code', 'CHROME_DESKTOP': 'code-url-handler.desktop', 'COLORTERM': 'truecolor', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus', 'DESKTOP_SESSION': 'ubuntu', 'DISPLAY': ':0', 'EDITOR': 'code', 'ELECTRON_NO_ATTACH_CONSOLE': '1', 'GDK_BACKEND': 'x11', 'GDMSESSION': 'ubuntu', 'GNOME_DESKTOP_SESSION_ID': 'this-is-deprecated', 'GNOME_SETUP_DISPLAY': ':1', 'GNOME_SHELL_SESSION_MODE': 'ubuntu', 'GNOME_TERMINAL_SCREEN': '/org/gnome/Terminal/screen/e92213ce_d4ff_4a49_ae11_4f76d98fd6f1', 'GNOME_TERMINAL_SERVICE': ':1.102', 'GTK_MODULES': 'gail:atk-bridge', 'HOME': '/home/hughharford', 'HOMEBREW_NO_ANALYTICS': '1', 'LANG': 'en_US.UTF-8', 'LANGUAGE': 'en_GB:en', 'LC_ALL': 'en_US.UTF-8', 'LESS': '-R', 'LOGNAME': 'hughharford', 'LSCOLORS': 'Gxfxcxdxbxegedabagacad', 'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*

# Imports
***

In [22]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload


In [8]:
import os
import time
            # data sourcing:
import csv
            # databases:
import sqlite3
            # useful other
from collections import defaultdict
            # data toolkit:
import numpy as np
import pandas as pd
import dask.dataframe as dd
            # plotting
import matplotlib.pyplot as plt
%matplotlib inline  

if not environment_running == 'laptop':
    %pip install seaborn
    # no seaborn by default in AWS SageMaker.
    
import seaborn as sns

In [10]:
import sys
sys.path.append("..") # this is only to enable imports in cell below...(!)

In [19]:
from util.minor_funcs import Minors
m_funcs = Minors()

In [34]:
# count no files & folders in /raw_data:
data_files_found = 0
data_files_found = m_funcs.get_list_raw_data_files()
print(len(data_files_found))

3


In [32]:
import os
directory = '../../raw_data/' 
onlyfiles = next(os.walk(directory))[2]

## NB: 
The below will hang if you don't have the data: use a manual download for now

In [17]:
# run this cell to retrieve data and place in the correct location for the below
# obviously, a login will be needed, or kaggle authentication in environment variable
retrieve_data = 0
if data_files_found == 0: retrieve_data = 1

if retrieve_data:
    !cd /raw_data/
    !kaggle datasets download -d ehallmar/a-large-tennis-dataset-for-atp-and-itf-betting
    
# alternatively:
# download to the raw_data folder from the following url
dataset_url = 'https://www.kaggle.com/datasets/ehallmar/a-large-tennis-dataset-for-atp-and-itf-betting?select=all_tournaments.csv'

data_files_found = count_raw_data()
assert len(data_files_found) > 0

In [18]:
# check raw data archive path extant:
atp_data_path = os.path.join('..','raw_data','archive')
assert os.path.exists(atp_data_path)

# set read_data path for chunking later on:
read_data_path = os.path.join('..','raw_data','read_data')
assert os.path.exists(read_data_path) 

In [19]:
# unzip archive, if not already done

# confirm .csv files x 7 are there:
assert os.path.isfile('../raw_data/archive/all_matches.csv'), 'all_matches.csv not found. download from url above'
assert os.path.isfile('../raw_data/archive/all_players.csv'), 'all_matches.csv not found. download from url above'
assert os.path.isfile('../raw_data/archive/all_tournaments.csv'), 'all_matches.csv not found. download from url above'
assert os.path.isfile('../raw_data/archive/betting_moneyline.csv'), 'all_matches.csv not found. download from url above'
assert os.path.isfile('../raw_data/archive/betting_spreads.csv'), 'all_matches.csv not found. download from url above'
assert os.path.isfile('../raw_data/archive/betting_totals.csv'), 'all_matches.csv not found. download from url above'
assert os.path.isfile('../raw_data/archive/countries.csv'), 'all_matches.csv not found. download from url above'


In [20]:
# check initial data file expected are extant:
matches_data_path_full = os.path.join(atp_data_path, 'all_matches.csv')
assert os.path.exists(matches_data_path_full)